<a href="https://colab.research.google.com/github/imji0319/PDSH/blob/master/NYC_modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd /content/gdrive/My Drive/data/nyc_split

/content/gdrive/My Drive/data/nyc_split


In [3]:
ls

nyc_0.feather   nyc_14.feather  nyc_19.feather  nyc_4.feather  nyc_9.feather
nyc_10.feather  nyc_15.feather  nyc_1.feather   nyc_5.feather
nyc_11.feather  nyc_16.feather  nyc_20.feather  nyc_6.feather
nyc_12.feather  nyc_17.feather  nyc_2.feather   nyc_7.feather
nyc_13.feather  nyc_18.feather  nyc_3.feather   nyc_8.feather


In [4]:
pip install -U pyarrow

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (0.13.0)


In [0]:
import pandas as pd
import numpy as np

In [0]:
# read data
df_train =pd.read_feather('nyc_0.feather')

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600000 entries, 0 to 2599999
Data columns (total 13 columns):
level_0               int64
index                 int64
fare_amount           float32
pickup_longitude      float32
pickup_latitude       float32
dropoff_longitude     float32
dropoff_latitude      float32
passenger_count       uint8
year                  int64
weekday               uint8
hour                  uint8
distance_miles        float32
distance_to_center    float32
dtypes: float32(7), int64(3), uint8(3)
memory usage: 136.4 MB


In [8]:
df_train.head()

,level_0,index,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,hour,distance_miles,distance_to_center
0,0,0,4.5,-73.844315,40.721317,-73.841614,40.712276,1,2009,0,17,0.640473,8.501885
1,1,1,16.9,-74.016045,40.711304,-73.979271,40.782005,1,2010,1,16,5.250587,0.543003
2,2,2,5.7,-73.982735,40.761269,-73.991241,40.750561,2,2011,3,0,0.863477,3.482071
3,3,3,7.7,-73.987129,40.733143,-73.991570,40.758091,1,2012,5,4,1.739349,1.654125
4,4,4,5.3,-73.968094,40.768009,-73.956657,40.783764,1,2010,1,7,1.242172,4.225934


In [0]:
df_train = df_train.drop(columns=['level_0', 'index'])

In [0]:
from sklearn.model_selection import train_test_split 
import xgboost as xgb

In [0]:
y = df_train['fare_amount']
train = df_train.drop(columns=['fare_amount'])

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, y, random_state = 100, test_size=0.25)

In [0]:
#Cross-validation
params = {
    # Parameters that we are going to tune.
    'max_depth': 8, #Result of tuning with CV
    'eta':.03, #Result of tuning with CV
    'subsample': 1, #Result of tuning with CV
    'colsample_bytree': 0.8, #Result of tuning with CV
    # Other parameters
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 1
}

In [0]:
def XGBmodel(x_train,x_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(x_train,label=y_train)
    matrix_test = xgb.DMatrix(x_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=300, verbose_eval=50,
                    early_stopping_rounds=10,evals=[(matrix_test,'test')])
    return model

In [15]:
%%time

model = XGBmodel(X_train,X_test,y_train,y_test,params)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	test-rmse:13.9874
Will train until test-rmse hasn't improved in 10 rounds.
[50]	test-rmse:4.66032
[100]	test-rmse:3.50513
[150]	test-rmse:3.38591
[200]	test-rmse:3.34419
[250]	test-rmse:3.32345
[299]	test-rmse:3.30282
CPU times: user 52min 40s, sys: 1.31 s, total: 52min 41s
Wall time: 27min 2s


In [0]:
from xgboost import Booster

In [0]:
model.save_model('xgb.model')

# Test Prediction 

In [18]:
cd /content/gdrive/My Drive/data

/content/gdrive/My Drive/data


In [0]:
df_test = pd.read_csv('cleaning_test.csv')

In [20]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 12 columns):
key                   9914 non-null object
pickup_datetime       9914 non-null object
pickup_longitude      9914 non-null float64
pickup_latitude       9914 non-null float64
dropoff_longitude     9914 non-null float64
dropoff_latitude      9914 non-null float64
passenger_count       9914 non-null int64
year                  9914 non-null int64
weekday               9914 non-null int64
hour                  9914 non-null int64
distance_miles        9914 non-null float64
distance_to_center    9914 non-null float64
dtypes: float64(6), int64(4), object(2)
memory usage: 929.5+ KB


In [21]:
df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,weekday,hour,distance_miles,distance_to_center
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,13,1.443607,3.841863
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,13,1.507044,1.084252
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44,-73.982524,40.751260,-73.979654,40.746139,1,2011,5,11,0.384398,2.851254
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12,-73.981160,40.767807,-73.990448,40.751635,1,2012,5,21,1.218529,3.934481
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12,-73.966046,40.789775,-73.988565,40.744427,1,2012,5,21,3.347514,5.634652


In [0]:
test_key = df_test['key']
x_pred = df_test.drop(columns=['key', 'pickup_datetime'])

In [0]:
#Predict from test set
prediction = model.predict(xgb.DMatrix(x_pred), ntree_limit = model.best_ntree_limit)

In [24]:
submission = pd.DataFrame({'key' : df_test.key,
                          'fare_amount': prediction},
                         columns =['key','fare_amount'])

submission.head(10)

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.953129
1,2015-01-27 13:08:24.0000003,10.966424
2,2011-10-08 11:53:44.0000002,4.682038
3,2012-12-01 21:12:12.0000002,7.821180
4,2012-12-01 21:12:12.0000003,13.966803
5,2012-12-01 21:12:12.0000005,9.619411
6,2011-10-06 12:10:20.0000001,5.013222
7,2011-10-06 12:10:20.0000003,48.895218
8,2011-10-06 12:10:20.0000002,11.280354
9,2014-02-18 15:22:20.0000002,6.859297


In [0]:
submission.to_csv('xgb_submission1.csv', index=False)